In [1]:
import os
from accountables.utils.accounting_data import ACCOUNT_RECEIPT_PRIORITY
from accounting.models.ledger import LEDGER_RECEIPT_PRIORITY

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

pd.set_option('display.max_rows', 500)

In [2]:
acc = Accountable.objects.get(pk='A6N-13N-62-O1^2015-01-01')
acc

<Accountable: A6N-13N-62-O1^2015-01-01>

In [3]:
objs_df=pd.DataFrame(Charge.objects.filter(account__in=[13802005, 13050505], concept__accountable=acc)\
                    .values('id', 'ledger', 'concept__accountable', 'account', 'account__name', 'concept__date', 'value'))
priority_df=objs_df.assign(priority=objs_df.apply(lambda x: ACCOUNT_RECEIPT_PRIORITY[x.account] + LEDGER_RECEIPT_PRIORITY[x.ledger[:2]], axis=1))
sorted_df=priority_df.sort_values(by=['concept__date', 'priority', 'value'])
sorted_df

,id,ledger,concept__accountable,account,account__name,concept__date,value,priority
0,2323,CA-00000040,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-01-01,1065000,10
10,3283,FV-00000001,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-01-01,-1065000,11
11,3284,FV-00000001,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-01-01,1065000,11
12,3286,RC-00000001,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-01-01,-1065000,12
1,2431,CA-00000094,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-02-01,1065000,10
13,3287,FV-00000002,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-02-01,-1065000,11
14,3288,FV-00000002,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-02-01,1065000,11
2,2541,CA-00000149,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-03-01,1065000,10
3,2565,CA-00000161,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-04-01,1065000,10
4,2735,CA-00000246,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-05-01,1065000,10


In [4]:
def GetChargeDue(charge_id):
    cha=Charge.objects.get(pk=charge_id)
    if cha.ledger.type.abreviation == 'CA':
        if Charge.objects.filter(account=cha.account, concept=cha.concept, ledger__type__abreviation='FV', value=-cha.value).exists():
            return 0
    elif cha.ledger.type.abreviation == 'FV' and cha.value < 0:
        if Charge.objects.filter(account=cha.account, concept=cha.concept, ledger__type__abreviation='CA', value=-cha.value).exists():
            return 0        
    elif cha.ledger.type.abreviation == 'FV' and cha.value > 0:
        receipt = Charge.objects.filter(account=cha.account, concept=cha.concept, ledger__type__abreviation='RC', value__lt=0).aggregate(receipt=Sum('value'))['receipt']
        return cha.value + receipt if receipt else cha.value
    else:
        return 0

In [5]:
def DueAge(_dueDate):
    bufferDate=_dueDate
    months=0
    days=0
    while bufferDate < datetime.date.today():
        if bufferDate + relativedelta(months=1) < datetime.date.today():
            months = months +1
        else:
            days=(datetime.date.today() - bufferDate).days
        bufferDate = bufferDate + relativedelta(months=1)
    return f'{months}, {days}'

In [6]:
due_df=sorted_df.assign(
    due_value=sorted_df.id.apply(lambda x: GetChargeDue(x)),
    due_age=sorted_df.concept__date.apply(lambda x: DueAge(x)))
due_df

,id,ledger,concept__accountable,account,account__name,concept__date,value,priority,due_value,due_age
0,2323,CA-00000040,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-01-01,1065000,10,0.0,"10, 1"
10,3283,FV-00000001,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-01-01,-1065000,11,0.0,"10, 1"
11,3284,FV-00000001,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-01-01,1065000,11,0.0,"10, 1"
12,3286,RC-00000001,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-01-01,-1065000,12,0.0,"10, 1"
1,2431,CA-00000094,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-02-01,1065000,10,0.0,"9, 1"
13,3287,FV-00000002,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-02-01,-1065000,11,0.0,"9, 1"
14,3288,FV-00000002,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-02-01,1065000,11,1065000.0,"9, 1"
2,2541,CA-00000149,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-03-01,1065000,10,NaN,"8, 1"
3,2565,CA-00000161,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-04-01,1065000,10,NaN,"7, 1"
4,2735,CA-00000246,A6N-13N-62-O1^2015-01-01,13802005,Arrendamientos Inmuebles,2022-05-01,1065000,10,NaN,"6, 1"


In [7]:
pending_df = due_df[due_df['due_value'] != 0].to_dict('records')
pending_df

[{'id': 1057,
  'ledger': 'FV-00000007',
  'concept__accountable': 'K1-17-37^2012-02-01',
  'account': 13050505,
  'account__name': 'Impuesto a la Venta',
  'concept__date': datetime.date(2021, 12, 1),
  'value': 273600,
  'due_value': 273600,
  'due_age': '11, 1'},
 {'id': 1143,
  'ledger': 'FV-00000047',
  'concept__accountable': 'K1-17-37^2012-02-01',
  'account': 13050505,
  'account__name': 'Impuesto a la Venta',
  'concept__date': datetime.date(2022, 1, 1),
  'value': 273600,
  'due_value': 273600,
  'due_age': '10, 1'},
 {'id': 1201,
  'ledger': 'FV-00000069',
  'concept__accountable': 'K1-17-37^2012-02-01',
  'account': 13050505,
  'account__name': 'Impuesto a la Venta',
  'concept__date': datetime.date(2022, 2, 1),
  'value': 288800,
  'due_value': 288800,
  'due_age': '9, 1'},
 {'id': 1375,
  'ledger': 'FV-00000146',
  'concept__accountable': 'K1-17-37^2012-02-01',
  'account': 13050505,
  'account__name': 'Impuesto a la Venta',
  'concept__date': datetime.date(2022, 3, 1),
 

In [14]:
acc_con=Accountable_Concept.objects.all()[508]
acc_con

<Accountable_Concept: Canon Mensual Arriendo Inmueble^2021-12-01_K1-17-37^2012-02-01>

In [18]:
Charge.objects.exclude(state=0).filter(account__in=[account for account in ACCOUNT_RECEIPT_PRIORITY.keys()], concept=acc_con, value__gt=0, ledger__type__abreviation='FV').aggregate(receivable=Sum('value'))

{'receivable': 1713600}

In [20]:
Charge.objects.exclude(state=0).filter(account__in=[account for account in ACCOUNT_RECEIPT_PRIORITY.keys()], concept=acc_con, value__lt=0, ledger__type__abreviation='RC').aggregate(received=Sum('value')) if Charge.objects.exclude(state=0).filter(account__in=[account for account in ACCOUNT_RECEIPT_PRIORITY.keys()], concept=acc_con, value__lt=0, ledger__type__abreviation='RC').exists() else 0

0